In [4]:
#!pip3 install pandas

In [71]:
#!pip3 install tqdm

In [87]:
import pandas as pd
from collections import defaultdict
import copy
from tqdm import tqdm
import random
import json

In [7]:
df = pd.read_csv("../data/persuade/persuade_2.0_human_scores_demo_id_github.csv", usecols=["full_text"])

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25996 entries, 0 to 25995
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   full_text  25996 non-null  object
dtypes: object(1)
memory usage: 203.2+ KB


In [10]:
df.describe()

,full_text
count,25996
unique,25996
top,Phones\n\nModern humans today are always on th...
freq,1


In [11]:
df.head()

,full_text
0,Phones\n\nModern humans today are always on th...
1,This essay will explain if drivers should or s...
2,Driving while the use of cellular devices\n\nT...
3,Phones & Driving\n\nDrivers should not be able...
4,Cell Phone Operation While Driving\n\nThe abil...


In [18]:
len(df["full_text"][0])

2029

In [27]:
len(df["full_text"][0].split())

379

In [19]:
documents = df["full_text"]

In [20]:
type(documents)

pandas.core.series.Series

In [23]:
vocabs = set()
for doc in documents:
    for char in doc:
        vocabs.add(char)

In [24]:
len(vocabs)

141

In [28]:
end_of_word = "</w>"
vocabs.add(end_of_word)

In [54]:
init_token_tup_freqs = defaultdict(int)

In [55]:
for document in documents:
    for word in document.split():
        token_tup = tuple(list(word) + [end_of_word])
        init_token_tup_freqs[token_tup] += 1

In [36]:
len(token_tup_freqs.keys())

149425

In [53]:
def get_token_pair_stats(token_tup_freqs):
    token_pair_stats = defaultdict(int)
    for token_tup, freq in token_tup_freqs.items():
        for i in range(len(token_tup) - 1):
            token_pair = (token_tup[i], token_tup[i+1])
            token_pair_stats[token_pair] += freq
    return token_pair_stats

In [64]:
def merge_2_tokens(token_pair_to_merge, token_tup_freqs):
    new_token = token_pair_to_merge[0] + token_pair_to_merge[1]

    new_token_tup_freqs = defaultdict(int)
    for token_tup, freq in token_tup_freqs.items():
        new_token_tup_as_li = []
        i = 0
        while i < len(token_tup):
            if i < len(token_tup) - 1 and token_tup[i] == token_pair_to_merge[0] and token_tup[i+1] == token_pair_to_merge[1]:
                new_token_tup_as_li.append(new_token)
                i += 2
            else:
                new_token_tup_as_li.append(token_tup[i])
                i += 1
        new_token_tup_freqs[tuple(new_token_tup_as_li)] = freq
    return new_token_tup_freqs, new_token

In [49]:
merged_token_pairs = []

In [75]:
target_num_new_tokens = 149425 // 2   # Increase by 50%
target_num_new_tokens = 20000

In [76]:
token_tup_freqs = copy.deepcopy(init_token_tup_freqs)    

In [77]:
for _ in tqdm(range(target_num_new_tokens)):
    token_pair_stats = get_token_pair_stats(token_tup_freqs)
    most_freq_token_pair = max(token_pair_stats, key=token_pair_stats.get)
    merged_token_pairs.append(most_freq_token_pair)
    token_tup_freqs, new_vocab = merge_2_tokens(most_freq_token_pair, token_tup_freqs)
    vocabs.add(new_vocab)

100%|█████████████████████████████████| 20000/20000 [47:15<00:00,  7.05it/s]


In [78]:
start = 124
n = 20
# Between 0 to 19999

for token_pair in merged_token_pairs[start:start+n]:
    print(f"({str(token_pair)}) => {token_pair[0] + token_pair[1]}")

(('c', 'ar')) => car
(('pe', 'op')) => peop
(('u', 'n')) => un
(('a', 'd')) => ad
(('ou', 't</w>')) => out</w>
(('s', 'e')) => se
(('c', 'l')) => cl
(('s', 'om')) => som
(('e', 'x')) => ex
(('s', 'o</w>')) => so</w>
(('t', 'a')) => ta
(('or', 'e</w>')) => ore</w>
(('s', 'a')) => sa
(('w', 'or')) => wor
(('s', 't</w>')) => st</w>
(('s', 'u')) => su
(('be', 'cause</w>')) => because</w>
(('u', 't</w>')) => ut</w>
(('e', '.</w>')) => e.</w>
(('wi', 'll</w>')) => will</w>


In [80]:
vocabs_li = sorted(list(vocabs))

In [84]:
import random

sampled_token = random.choice(vocabs_li)
while len(sampled_token) < 2:
    sampled_token = random.choice(vocabs_li)
print(sampled_token)

rai


In [88]:
output_file = "../dist/tokens_with_end-of-word.json"

json_dict = {}
for i, vocab in enumerate(vocabs_li):
    json_dict[vocab] = i

with open(output_file, 'w') as json_file:
    json.dump(json_dict, json_file, indent=4)